In [1]:
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import gdown
import shutil
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# download files from gdrive

In [2]:
%%time
gdown.download('https://drive.google.com/file/d/1RjFrBLyNTjzYN9ow8HVfQFl3CYcGz8qM/view?usp=sharing', 
               quiet=False,fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1RjFrBLyNTjzYN9ow8HVfQFl3CYcGz8qM
From (redirected): https://drive.google.com/uc?id=1RjFrBLyNTjzYN9ow8HVfQFl3CYcGz8qM&confirm=t&uuid=343d5b94-6bfa-48f2-819b-b494e3daa524
To: /Volumes/T7/dataset.zip
100%|██████████████████████████████████████| 1.61G/1.61G [03:20<00:00, 8.05MB/s]

CPU times: user 8.34 s, sys: 9.72 s, total: 18.1 s
Wall time: 3min 22s


'dataset.zip'

# unzip files

In [3]:
%%time
shutil.unpack_archive('dataset.zip', '')
os.remove('dataset.zip')

CPU times: user 6.22 s, sys: 5.8 s, total: 12 s
Wall time: 16.5 s


# dataset

In [4]:
class FoodDataset(Dataset):

    def __init__(self, path_to_csv: str, path_to_imgs: str, is_read_all: bool, scale: int):
        self.nutr_info = pd.read_csv(path_to_csv)
        self.path_to_imgs = path_to_imgs
        self.is_read_all = is_read_all
        self.scale = scale
        if self.is_read_all:
            self.images = [self.transform(Image.open(f'{self.path_to_imgs}/{i}.png')) 
                           for i in self.nutr_info.img]

    def __len__(self):
        return len(self.self.nutr_info)
    
    def transform(self, img):
        size = min(img.size)
        return transforms.Compose([transforms.CenterCrop(size), 
                                   transforms.Resize(self.scale), 
                                   transforms.ToTensor()])(img)
    
    def __getitem__(self, idx):
        sample = dict(self.nutr_info[['kcal_total', 'mass', 'prot_total', 
                                      'fat_total', 'carb_total']].iloc[idx])
        if self.is_read_all:
            image = self.images[idx]
        else:
            image = self.transform(Image.open(
                f'{self.path_to_imgs}/{self.nutr_info.img.iloc[idx]}.png'))
        sample['image'] = image
        return sample

In [5]:
%%time
fd = FoodDataset(path_to_csv='dataset/train/train.csv', path_to_imgs='dataset/train', is_read_all=False, scale=64)
fd[300]

CPU times: user 27.6 ms, sys: 7.92 ms, total: 35.6 ms
Wall time: 249 ms


{'kcal_total': 4324.0,
 'mass': 1188.0,
 'prot_total': 63.0,
 'fat_total': 244.7,
 'carb_total': 465.7,
 'image': tensor([[[0.1686, 0.1490, 0.1608,  ..., 0.2549, 0.2431, 0.2314],
          [0.2078, 0.1843, 0.1882,  ..., 0.2471, 0.2392, 0.2314],
          [0.1961, 0.1843, 0.1843,  ..., 0.2510, 0.2510, 0.2353],
          ...,
          [0.8667, 0.8118, 0.7412,  ..., 0.8706, 0.8706, 0.8510],
          [0.8941, 0.8980, 0.8745,  ..., 0.8627, 0.8627, 0.8549],
          [0.8863, 0.8863, 0.8902,  ..., 0.8549, 0.8510, 0.8549]],
 
         [[0.1333, 0.0902, 0.0902,  ..., 0.2627, 0.2667, 0.2706],
          [0.1765, 0.1333, 0.1333,  ..., 0.2627, 0.2627, 0.2627],
          [0.1647, 0.1373, 0.1333,  ..., 0.2745, 0.2627, 0.2627],
          ...,
          [0.9608, 0.8745, 0.7922,  ..., 0.9725, 0.9725, 0.9608],
          [0.9843, 0.9686, 0.9333,  ..., 0.9765, 0.9765, 0.9686],
          [0.9922, 0.9882, 0.9843,  ..., 0.9804, 0.9804, 0.9725]],
 
         [[0.1765, 0.1294, 0.1255,  ..., 0.3412, 0.3333, 0.

In [6]:
%%time
fd = FoodDataset(path_to_csv='dataset/train/train.csv', path_to_imgs='dataset/train', is_read_all=True, scale=64)
fd[300]

CPU times: user 29.4 s, sys: 818 ms, total: 30.2 s
Wall time: 42 s


{'kcal_total': 4324.0,
 'mass': 1188.0,
 'prot_total': 63.0,
 'fat_total': 244.7,
 'carb_total': 465.7,
 'image': tensor([[[0.1686, 0.1490, 0.1608,  ..., 0.2549, 0.2431, 0.2314],
          [0.2078, 0.1843, 0.1882,  ..., 0.2471, 0.2392, 0.2314],
          [0.1961, 0.1843, 0.1843,  ..., 0.2510, 0.2510, 0.2353],
          ...,
          [0.8667, 0.8118, 0.7412,  ..., 0.8706, 0.8706, 0.8510],
          [0.8941, 0.8980, 0.8745,  ..., 0.8627, 0.8627, 0.8549],
          [0.8863, 0.8863, 0.8902,  ..., 0.8549, 0.8510, 0.8549]],
 
         [[0.1333, 0.0902, 0.0902,  ..., 0.2627, 0.2667, 0.2706],
          [0.1765, 0.1333, 0.1333,  ..., 0.2627, 0.2627, 0.2627],
          [0.1647, 0.1373, 0.1333,  ..., 0.2745, 0.2627, 0.2627],
          ...,
          [0.9608, 0.8745, 0.7922,  ..., 0.9725, 0.9725, 0.9608],
          [0.9843, 0.9686, 0.9333,  ..., 0.9765, 0.9765, 0.9686],
          [0.9922, 0.9882, 0.9843,  ..., 0.9804, 0.9804, 0.9725]],
 
         [[0.1765, 0.1294, 0.1255,  ..., 0.3412, 0.3333, 0.

In [8]:
transforms.Compose([transforms.CenterCrop(208), 
                    transforms.Resize(64), 
                    transforms.ToTensor()])(Image.open(
    f'dataset/train/{pd.read_csv("dataset/train/train.csv").img.iloc[300]}.png'))

tensor([[[0.1686, 0.1490, 0.1608,  ..., 0.2549, 0.2431, 0.2314],
         [0.2078, 0.1843, 0.1882,  ..., 0.2471, 0.2392, 0.2314],
         [0.1961, 0.1843, 0.1843,  ..., 0.2510, 0.2510, 0.2353],
         ...,
         [0.8667, 0.8118, 0.7412,  ..., 0.8706, 0.8706, 0.8510],
         [0.8941, 0.8980, 0.8745,  ..., 0.8627, 0.8627, 0.8549],
         [0.8863, 0.8863, 0.8902,  ..., 0.8549, 0.8510, 0.8549]],

        [[0.1333, 0.0902, 0.0902,  ..., 0.2627, 0.2667, 0.2706],
         [0.1765, 0.1333, 0.1333,  ..., 0.2627, 0.2627, 0.2627],
         [0.1647, 0.1373, 0.1333,  ..., 0.2745, 0.2627, 0.2627],
         ...,
         [0.9608, 0.8745, 0.7922,  ..., 0.9725, 0.9725, 0.9608],
         [0.9843, 0.9686, 0.9333,  ..., 0.9765, 0.9765, 0.9686],
         [0.9922, 0.9882, 0.9843,  ..., 0.9804, 0.9804, 0.9725]],

        [[0.1765, 0.1294, 0.1255,  ..., 0.3412, 0.3333, 0.3216],
         [0.2314, 0.1804, 0.1686,  ..., 0.3333, 0.3255, 0.3137],
         [0.2196, 0.1843, 0.1725,  ..., 0.3412, 0.3216, 0.